<a href="https://colab.research.google.com/github/BenyaminZojaji/Deep_Learning/blob/main/PyTorch%20FashionMNIST/FashionMnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

In [4]:
import torch
import torchvision
from tqdm import tqdm
import wandb

In [ ]:
run = wandb.init(project="pytorch-fashionMnist", entity="benyaminzojaji")

In [48]:
def calc_acc(preds, labels):
  _, preds_max = torch.max(preds, 1)
  acc = torch.sum(preds_max == labels.data, dtype=torch.float64) / len(preds)
  return acc

In [49]:
class MyModel(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.fc1 = torch.nn.Linear(28*28, 128)
    self.fc2 = torch.nn.Linear(128, 256)
    self.fc3 = torch.nn.Linear(256, 512)
    self.fc4 = torch.nn.Linear(512, 10)

  def forward(self, x):
    # input shape: 32 * 28 * 28 * 1 <-> batch-size * height * width * channel
    x = x.reshape((x.shape[0], 784))
    # output shape: 32 * 784

    x = self.fc1(x)
    x = torch.relu(x)
    x = torch.dropout(x, 0.3, train=True)
    x = self.fc2(x)
    x = torch.relu(x)
    x = torch.dropout(x, 0.3, train=True)
    x = self.fc3(x)
    x = torch.relu(x)
    x = torch.dropout(x, 0.3, train=True)
    x = self.fc4(x)
    x = torch.softmax(x, dim=1)
    return x

In [ ]:
device = torch.device('cuda')
# device = torch.device('cpu')

model = MyModel()
model = model.to(device)
model.train(True)

In [11]:
# Data Preparing

transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0), (1))
])

In [51]:
batch_size = 64
lr = 0.0001
epochs = 10

In [12]:
!mkdir dataset

In [14]:
dataset = torchvision.datasets.FashionMNIST('./dataset', train=True, download=True, transform=transform)
train_data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [20]:
dataset = torchvision.datasets.FashionMNIST('./dataset', train=False, download=True, transform=transform)
test_data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [52]:
#
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_function = torch.nn.CrossEntropyLoss()

In [53]:
# train

def train_step(images, labels):
  loss = 0.0
  acc = 0.0

  images = images.to(device)
  labels = labels.to(device)
  optimizer.zero_grad()

  # 1- forwarding
  preds = model(images)

  # 2- backwarding
  loss = loss_function(preds, labels)
  loss.backward()

  # 3- Update
  optimizer.step()

  acc = calc_acc(preds, labels)

  return loss, acc

In [54]:
def test_step(images, labels):
  loss = 0.0
  acc = 0.0

  images = images.to(device)
  labels = labels.to(device)

  preds = model(images)

  loss = loss_function(preds, labels)

  acc = calc_acc(preds, labels)

  return loss, acc

In [55]:
def train():
  for epoch in range(epochs):
    train_loss= 0.0
    train_acc = 0.0
    test_loss = 0.0
    test_acc = 0.0

    for images, labels in tqdm(train_data_loader):
      t_loss, t_acc = train_step(images, labels)
      train_loss += t_loss
      train_acc += t_acc
    total_train_loss = train_loss / len(train_data_loader)
    total_train_acc = train_acc / len(train_data_loader)

    for images, labels in tqdm(test_data_loader):
      t_loss, t_acc = test_step(images, labels)
      test_loss += t_loss
      test_acc += t_acc
    total_test_loss = test_loss / len(test_data_loader)
    total_test_acc = test_acc / len(test_data_loader)
  
    print(f'Epoch {epoch+1}\nTrain-Acc: {total_train_acc}, Train-Loss: {total_train_loss}\nTest-acc: {total_test_acc}, Test-Loss: {total_test_loss}')


    wandb.log({'epochs': epoch+1,
              'train_loss': total_train_loss,
              'train_accuracy': float(total_train_acc),
              'test_loss': total_test_loss,
              'test_accuracy': float(total_test_acc)
              })

In [56]:
train()
run.finish()

100%|██████████| 157/157 [00:02<00:00, 61.57it/s]


Epoch 1
Train-Acc: 0.6064598880597015, Train-Loss: 1.875532627105713
Test-acc: 0.7395501592356688, Test-Loss: 1.737741231918335


100%|██████████| 157/157 [00:02<00:00, 60.81it/s]


Epoch 2
Train-Acc: 0.76554171108742, Train-Loss: 1.7048277854919434
Test-acc: 0.7760748407643312, Test-Loss: 1.694515347480774


100%|██████████| 157/157 [00:02<00:00, 54.41it/s]


...

100%|██████████| 157/157 [00:02<00:00, 59.00it/s]


Epoch 9
Train-Acc: 0.8494469616204691, Train-Loss: 1.6138088703155518
Test-acc: 0.8387738853503185, Test-Loss: 1.6237313747406006


100%|██████████| 157/157 [00:02<00:00, 61.38it/s]


Epoch 10
Train-Acc: 0.8525119936034115, Train-Loss: 1.6104018688201904
Test-acc: 0.8408638535031847, Test-Loss: 1.62169349193573



epochs,▁▂▃▃▄▅▆▆▇█
test_accuracy,▁▄▅▆▆▇▇███
test_loss,█▅▄▃▂▂▂▁▁▁
train_accuracy,▁▆▆▇▇▇████
train_loss,█▃▃▂▂▂▁▁▁▁
epochs,10
test_accuracy,0.84086
test_loss,1.62169
train_accuracy,0.85251
train_loss,1.6104


In [57]:
# save
torch.save(model.state_dict(), 'FashionMNIST.pth')

In [60]:
# inference

import cv2
import numpy as np
import time

#model.train(False)
model.eval()

# oreoricess
img = cv2.imread('/content/pullover.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (28, 28))
tensor = transform(img).unsqueeze(0).to(device)

# process
start = time.time()
preds = model(tensor)
end = time.time()

# postprocess
# _, output=torch.max(preds, 1)
preds = preds.cpu().detach().numpy()
output = np.argmax(preds)
classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
print(f'{classes[output]}, Process-Time: {end-start}')

Pullover, Process-Time: 0.001168966293334961
